# Waste Distribution

In [ ]:
import matplotlib.pyplot as plt
import geopandas as gpd
import pandas as pd
import os

## Load data

In [ ]:
onedrive = os.path.join("/", "mnt", "c", "Users", "eschmann", "OneDrive - epfl.ch", "Research IT", "Advanced Services", "0042 – Blue City", "BlueCityViz")
SP04 = os.path.join(onedrive, "SP04_Waste")
waste_dir = os.path.join(SP04, "Waste collection points by types (density-based spatial clustering)")

sous_secteurs_filename = os.path.join(SP04, "Sous-secteurs Lausanne", "Sous-secteurs.shp")
Household_waste_filename = os.path.join(waste_dir, "DI_final_clustered_centroids.csv")
paper_waste_filename = os.path.join(waste_dir, "PC_final_clustered_centroids.csv")
glass_waste_filename = os.path.join(waste_dir, "VE_final_clustered_centroids.csv")
Organic_waste_filename = os.path.join(waste_dir, "DV_final_clustered_centroids.csv")

sous_secteurs = gpd.read_file(sous_secteurs_filename)

Household_waste = gpd.read_file(Household_waste_filename)
paper_waste = gpd.read_file(paper_waste_filename)
glass_waste = gpd.read_file(glass_waste_filename)
Organic_waste = gpd.read_file(Organic_waste_filename)

In [ ]:
def create_geometry(df):
    """
    Create a geometry column from the longitude and latitude columns of a DataFrame.
    """
    df["geometry"] = gpd.points_from_xy(df["centroid_lon"], df["centroid_lat"])

    df.drop(columns=["centroid_lon", "centroid_lat"], inplace=True)

    return df


## Create a map of the waste collection points in Lausanne, colored by type of waste.

In [ ]:
Household_waste = create_geometry(Household_waste)
paper_waste = create_geometry(paper_waste)
glass_waste = create_geometry(glass_waste)
Organic_waste = create_geometry(Organic_waste)

Household_waste["type"] = "DI"
paper_waste["type"] = "PC"
glass_waste["type"] = "VE"
Organic_waste["type"] = "DV"

total_waste = pd.concat([Household_waste, paper_waste, glass_waste, Organic_waste], ignore_index=True)

total_waste = gpd.GeoDataFrame(total_waste, geometry="geometry", crs="EPSG:4326")
total_waste.to_crs(sous_secteurs.crs, inplace=True)

## Join the waste collection points with the sous-secteurs.

In [ ]:
waste_by_type = sous_secteurs.sjoin(total_waste, how="left", predicate="contains")
final_map = waste_by_type.groupby([waste_by_type.index, 'type']).size().unstack(fill_value=0)
final_map = sous_secteurs.join(final_map)

## Plot final result

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(15, 15))
axes = axes.flatten()

waste_types = [
    ('DI', 'Household Waste'),
    ('PC', 'Paper/Cardboard'),
    ('VE', 'Glass'),
    ('DV', 'Organic Waste')
]

for idx, (col, title) in enumerate(waste_types):
    ax = axes[idx]
    final_map.plot(column=col, ax=ax, legend=True, 
                   cmap='YlOrRd', edgecolor='black', linewidth=0.5)
    ax.set_title(f'{title} ({col})', fontsize=14)
    ax.set_axis_off()

plt.tight_layout()
plt.show()